In [1]:
from pyspark.sql import SparkSession, functions as F

In [2]:
spark = SparkSession \
        .builder.appName("ETL") \
        .getOrCreate()


In [4]:
df_exportacao = spark.read.json("../data/exportacao.json")
df_producao = spark.read.json("../data/producao.json")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/kleryton/Desktop/POS_Fiap/api-etl-pipeline/etl/data/exportacao.json.

In [25]:
df_exportacao.show()

+----+--------------+----------+--------------------+-------------+--------+
| ANO|     CATEGORIA|    ORIGEM|              PAISES|QUANTIDADE_KG|VALOR_US|
+----+--------------+----------+--------------------+-------------+--------+
|1970|Vinhos de mesa|EXPORTACAO|         Afeganistão|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|       África do Sul|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|Alemanha, Repúbli...|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|              Angola|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|            Anguilla|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|   Antígua e Barbuda|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO| Antilhas Holandesas|        280.0|   207.0|
|1970|Vinhos de mesa|EXPORTACAO|      Arábia Saudita|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|           Argentina|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|               Aruba|          0.0|     0.0|

In [26]:

df_exportacao.show()

+----+--------------+----------+--------------------+-------------+--------+
| ANO|     CATEGORIA|    ORIGEM|              PAISES|QUANTIDADE_KG|VALOR_US|
+----+--------------+----------+--------------------+-------------+--------+
|1970|Vinhos de mesa|EXPORTACAO|         Afeganistão|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|       África do Sul|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|Alemanha, Repúbli...|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|              Angola|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|            Anguilla|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|   Antígua e Barbuda|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO| Antilhas Holandesas|        280.0|   207.0|
|1970|Vinhos de mesa|EXPORTACAO|      Arábia Saudita|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|           Argentina|          0.0|     0.0|
|1970|Vinhos de mesa|EXPORTACAO|               Aruba|          0.0|     0.0|

In [27]:


df_exportacao = df_exportacao.select("PAISES", "QUANTIDADE_KG", "VALOR_US", "ANO").distinct() \
    .groupBy("PAISES", "ANO") \
    .agg(
        F.sum("QUANTIDADE_KG").alias("Quantidade (kg)"),
        F.sum("VALOR_US").alias("Valor (US$)")
    ) \
    .withColumnRenamed("ANO", "Ano") \
    .withColumnRenamed("PAISES", "Pais") \
    .orderBy("ANO") \
    .filter((F.col("Quantidade (kg)") > 0) & (F.col("Valor (US$)") > 0))

df_exportacao.show()


+--------------------+----+---------------+-----------+
|                Pais| Ano|Quantidade (kg)|Valor (US$)|
+--------------------+----+---------------+-----------+
| Antilhas Holandesas|1970|          280.0|      207.0|
|      Estados Unidos|1970|        11200.0|     4200.0|
|              Guiana|1970|          114.0|       36.0|
|           Outros(1)|1970|         5132.0|     3208.0|
|            Tanzânia|1970|        13692.0|     3562.0|
|             Libéria|1970|         4160.0|     1190.0|
|             Bolívia|1970|         2548.0|      699.0|
|            Paraguai|1970|        90261.0|    24132.0|
|            Colômbia|1970|        14205.0|     6650.0|
|Alemanha, Repúbli...|1971|          135.0|      103.0|
|      Estados Unidos|1971|        22400.0|     8400.0|
|             Bolívia|1971|         9450.0|     2908.0|
|              França|1971|          220.0|      110.0|
|  Trinidade e Tobago|1971|         7896.0|     2579.0|
| Antilhas Holandesas|1971|         4800.0|     

In [28]:
df_producao = df_producao.select("PRODUTO", "QUANTIDADE_L", "ANO").distinct() \
    .withColumnRenamed("PRODUTO", "produto") \
    .withColumnRenamed("QUANTIDADE_L", "quantidade (Litros)") \
    .withColumnRenamed("ANO", "Ano") \
    .groupBy("produto", "ANO") \
    .agg(
        F.sum("quantidade (Litros)").alias("Total_quantidade (Litros)")
    ) \
    .filter((F.col("Total_quantidade (Litros)") > 0))\
    .orderBy("ANO")

df_producao.show()


+--------------------+----+-------------------------+
|             produto| Ano|Total_quantidade (Litros)|
+--------------------+----+-------------------------+
|           Bagaceira|1970|                 961290.0|
|           Licorosos|1970|                 282500.0|
|       Mosto simples|1970|                7170736.0|
|           Destilado|1970|                 830030.0|
|       Borra líquida|1970|                3847673.0|
|Suco de uva integral|1970|                1097771.0|
|              Rosado|1970|              4.2981052E7|
|               Tinto|1970|             1.81815609E8|
|              Branco|1970|              1.6311289E7|
|            Mistelas|1970|                1072100.0|
|               Tinto|1971|             1.28399035E8|
|       Borra líquida|1971|                3565644.0|
|           Compostos|1971|                  45000.0|
|            Mistelas|1971|                 818400.0|
|           Licorosos|1971|                 270000.0|
|              Rosado|1971| 

In [88]:
df_producao_pandas = df_producao.toPandas()
df_exportacao_pandas = df_exportacao.toPandas()
df_producao_pandas.to_csv("../output/dados_producao.csv", index=False, header=True)
df_exportacao_pandas.to_csv("../output/dados_exportacao.csv", index=False, header=True)